1. 문제정의

- 주어진 데이터 train을 통해 식용인지 독버섯인지 구분
- DecisionT(tree 구조 모델 활용)
- 특성의 중요도 확인(특성선택;feature selection) 사용
- 주어진 데이터를 통해 모델 학습한 뒤, 독버섯인지 식용버섯인지 구분하는 문제
- train, test데이터가 따로 구분되어 있지 않다.

2. 라이브러리 호출

In [1]:
# 환경 준비
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import metrics
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
# 트리 모델 불러오기
from sklearn.tree import DecisionTreeClassifier
# 의사결정 트리 분류모델: 독버섯인지 식용인지 -> 1인지 0인지로 분류
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score #과대적합 제어 도구
from sklearn import tree
from sklearn.preprocessing import LabelEncoder

C:\Users\82109\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# 데이터 불러오기
# 출력 되는 컬럼의 개수 설정
pd.set_option('display.max_columns',None)
mush = pd.read_csv('mushrooms.csv')
mush.head(5) # 5개 확인

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,stalk-root,stalk-surface-above-ring,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,e,e,s,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,e,c,s,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,e,c,s,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,e,e,s,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,t,e,s,s,w,w,p,w,o,e,n,a,g


모든 값이 object 타입인걸 확인할 수 있다.

In [3]:
#data 전체 크기 확인
mush.shape

(8124, 23)

In [4]:
#data 전체 정보 출력 & 결측치 확인
mush.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8124 entries, 0 to 8123
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   class                     8124 non-null   object
 1   cap-shape                 8124 non-null   object
 2   cap-surface               8124 non-null   object
 3   cap-color                 8124 non-null   object
 4   bruises                   8124 non-null   object
 5   odor                      8124 non-null   object
 6   gill-attachment           8124 non-null   object
 7   gill-spacing              8124 non-null   object
 8   gill-size                 8124 non-null   object
 9   gill-color                8124 non-null   object
 10  stalk-shape               8124 non-null   object
 11  stalk-root                8124 non-null   object
 12  stalk-surface-above-ring  8124 non-null   object
 13  stalk-surface-below-ring  8124 non-null   object
 14  stalk-color-above-ring  

In [5]:
encoding = LabelEncoder()
for col in mush.columns:
  mush[col] = encoding.fit_transform(mush[col])
mush.head()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,stalk-root,stalk-surface-above-ring,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,1,5,2,4,1,6,1,0,1,4,0,3,2,2,7,7,0,2,1,4,2,3,5
1,0,5,2,9,1,0,1,0,0,4,0,2,2,2,7,7,0,2,1,4,3,2,1
2,0,0,2,8,1,3,1,0,0,5,0,2,2,2,7,7,0,2,1,4,3,2,3
3,1,5,3,8,1,6,1,0,1,5,0,3,2,2,7,7,0,2,1,4,2,3,5
4,0,5,2,3,0,5,1,1,0,4,1,3,2,2,7,7,0,2,1,0,3,0,1


target 데이터와 feature 데이터를 분리하고 target데이터가 0과1로 되어있는지 확인

In [6]:
target = mush['class']
feature = mush.drop('class',axis = 1)
target.unique()

array([1, 0])

In [7]:
model = DecisionTreeClassifier()
model2 = LogisticRegression()
x_train,x_test,y_train,y_test = train_test_split(feature,target,test_size = 0.3,random_state = 0)
model.fit(x_train,y_train)
model2.fit(x_train,y_train)
pred = model.predict(x_test)
pred2 = model2.predict(x_test)
print(accuracy_score(y_test,pred))
print(accuracy_score(y_test,pred2))

1.0
0.9495488105004102


C:\Users\82109\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


DecisionTree 정확도: 1.0
LogisticRegression정확도: 0.9491386382280558
LogisticRegession 보다 DecisionTree의 정확도가 더 높다

In [8]:
# 문제, 답 데이터 분리
# 문제 데이터
X = mush.iloc[:,1:]
# 답 데이터 >> poisonous(인덱스 0번 값)
y = mush.iloc[:, 0]

print(X.shape)
print(y.shape)

(8124, 22)
(8124,)


In [9]:
#데이터의 통계값 확인
mush.describe()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,stalk-root,stalk-surface-above-ring,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
count,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.0,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000
mean,0.482029,3.348104,1.827671,4.504677,0.415559,4.144756,0.974151,0.161497,0.309207,4.810684,0.567208,1.109798,1.575086,1.603644,5.816347,5.794682,0.0,1.965534,1.069424,2.291974,3.596750,3.644018,1.508616
std,0.499708,1.604329,1.229873,2.545821,0.492848,2.103729,0.158695,0.368011,0.462195,3.540359,0.495493,1.061106,0.621459,0.675974,1.901747,1.907291,0.0,0.242669,0.271064,1.801672,2.382663,1.252082,1.719975
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,0.000000,3.000000,0.000000,2.000000,1.000000,0.000000,0.000000,2.000000,0.000000,0.000000,1.000000,1.000000,6.000000,6.000000,0.0,2.000000,1.000000,0.000000,2.000000,3.000000,0.000000
50%,0.000000,3.000000,2.000000,4.000000,0.000000,5.000000,1.000000,0.000000,0.000000,5.000000,1.000000,1.000000,2.000000,2.000000,7.000000,7.000000,0.0,2.000000,1.000000,2.000000,3.000000,4.000000,1.000000
75%,1.000000,5.000000,3.000000,8.000000,1.000000,5.000000,1.000000,0.000000,1.000000,7.000000,1.000000,1.000000,2.000000,2.000000,7.000000,7.000000,0.0,2.000000,1.000000,4.000000,7.000000,4.000000,2.000000
max,1.000000,5.000000,3.000000,9.000000,1.000000,8.000000,1.000000,1.000000,1.000000,11.000000,1.000000,4.000000,3.000000,3.000000,8.000000,8.000000,0.0,3.000000,2.000000,4.000000,8.000000,5.000000,6.000000


In [10]:
# 답 데이터 균형 확인
y.value_counts()

0    4208
1    3916
Name: class, dtype: int64

답 2개의 개수가 비슷하므로 테스트 하기 적합한 데이터로 판단할 수 있다.

In [12]:
# 훈련 데이터, 테스트 데이터 분리 >> 7:3의 비율을 맞춘다.
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state =1, test_size=0.3)

In [13]:
# 모델 객체 정의
tree_model = DecisionTreeClassifier() #결정트리 객체
tree_model.fit(X_train,y_train) # 훈련데이터 학습
cross_val_score(tree_model, X_train, y_train, cv=5) # cv는 교차검증 진행 횟수
# 테스트 데이터의 비율에 따라 데이터의 민감도를 측정한다

array([1., 1., 1., 1., 1.])

In [14]:
# 70%로 나눠진 데이터들로 생성된 모델을 학습
tree_model.fit(x_train, y_train) # 해당과정 시간이 많이 걸림...

DecisionTreeClassifier()

In [15]:
# 크기 확인
print('훈련용 문제 : ', X_train.shape)
print('훈련용 답 : ',y_train.shape)
print('테스트용 문제 : ',X_test.shape)
print('테스트용 답 : ',y_test.shape)

훈련용 문제 :  (5686, 22)
훈련용 답 :  (5686,)
테스트용 문제 :  (2438, 22)
테스트용 답 :  (2438,)


In [16]:
# 70%로 나눠진 데이터들로 생성된 모델을 학습
tree_model.fit(X_train, y_train) # 해당과정 시간이 많이 걸림...

DecisionTreeClassifier()

In [17]:
tree_model.predict(X_test)

array([0, 1, 1, ..., 1, 0, 1])

In [18]:
# 학습시킨 모델에 실제 값이라 가정된 x_test 데이터들을 대입. 
pre=tree_model.predict(X_test)

In [19]:
# 정확도 확인
accuracy_score(pre,y_test)

1.0

In [24]:
# 과대적합 제어
tree_model2=DecisionTreeClassifier(max_depth=3)

# 과대 적합 제어 후 모델의 일반화 성능 확인
cross_val_score(tree_model2, x_train, y_train, cv=5)
# 5번 모델을 테스트해봐도 교차검증 정확도에 큰 차이가 없었다.
# 그러므로 이 모델은 일반화가 된 모델일 가능성이 높다

array([0.52724077, 0.51978892, 0.54529464, 0.5294635 , 0.51978892])

In [25]:
ip = tree_model.feature_importances_
ip

array([0.        , 0.        , 0.00323516, 0.03036548, 0.        ,
       0.        , 0.        , 0.12618094, 0.3414692 , 0.02709067,
       0.05422084, 0.        , 0.00289943, 0.        , 0.01763466,
       0.        , 0.00333916, 0.00122615, 0.        , 0.20207511,
       0.17476312, 0.01550008])

In [20]:
# 평가
tree_model.score(X_train,y_train)

1.0

In [21]:
# 평가 데이터로의 평가
tree_model.score(X_test,y_test)

1.0

과대적합 발생

In [22]:
tree_model_result = cross_val_score(tree_model,X_train,y_train,cv=5)
#cv=교차검증 횟수 지정
print("교차검증 결과: ", tree_model_result)
print("교차검증 결과 평균값: ", tree_model_result.mean())

교차검증 결과:  [1. 1. 1. 1. 1.]
교차검증 결과 평균값:  1.0
